## Setup and Imports

In [ ]:
import requests
import json
import pandas as pd
from typing import Dict, Any

# API Configuration
API_URL = "http://localhost:8000/graphql"
HEALTH_URL = "http://localhost:8000/health"
STATS_URL = "http://localhost:8000/stats"

print("✅ Imports successful!")

## Helper Functions

In [ ]:
def run_graphql_query(query: str, variables: Dict[str, Any] = None) -> Dict:
    """
    Execute a GraphQL query against the API
    """
    payload = {
        "query": query,
        "variables": variables or {}
    }
    
    response = requests.post(API_URL, json=payload)
    
    if response.status_code == 200:
        result = response.json()
        if "errors" in result:
            print("❌ GraphQL Errors:")
            print(json.dumps(result["errors"], indent=2))
        return result
    else:
        print(f"❌ HTTP Error {response.status_code}")
        print(response.text)
        return None

def display_branches(data: Dict) -> pd.DataFrame:
    """
    Convert branches data to pandas DataFrame for better visualization
    """
    if not data or "data" not in data or "branches" not in data["data"]:
        return pd.DataFrame()
    
    edges = data["data"]["branches"]["edges"]
    branches = []
    
    for edge in edges:
        node = edge["node"]
        branches.append({
            "IFSC": node.get("ifsc", ""),
            "Branch": node.get("branch", ""),
            "Bank": node.get("bank", {}).get("name", ""),
            "City": node.get("city", ""),
            "District": node.get("district", ""),
            "State": node.get("state", "")
        })
    
    return pd.DataFrame(branches)

print("✅ Helper functions defined!")

## Test 1: Health Check

In [ ]:
# Check if API is running
response = requests.get(HEALTH_URL)
if response.status_code == 200:
    health = response.json()
    print("✅ API is healthy!")
    print(json.dumps(health, indent=2))
else:
    print("❌ API is not responding. Make sure it's running!")

## Test 2: API Statistics

In [ ]:
# Get database statistics
response = requests.get(STATS_URL)
if response.status_code == 200:
    stats = response.json()
    print("📊 API Statistics:")
    print(f"  - Total Banks: {stats.get('banks', 0):,}")
    print(f"  - Total Branches: {stats.get('branches', 0):,}")
    print(f"  - API Version: {stats.get('api_version', 'N/A')}")

## Test 3: Query All Branches (Basic)

In [ ]:
# Basic query - get first 10 branches
query = """
query {
  branches(first: 10) {
    edges {
      node {
        ifsc
        branch
        bank {
          name
        }
        city
        state
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    total = result["data"]["branches"]["totalCount"]
    print(f"\n✅ Query successful! Total branches: {total:,}\n")
    
    # Display as DataFrame
    df = display_branches(result)
    display(df)

## Test 4: Query with Pagination

In [ ]:
# Query with pagination info
query = """
query {
  branches(first: 5) {
    edges {
      node {
        ifsc
        branch
        bank {
          name
        }
        city
      }
      cursor
    }
    pageInfo {
      hasNextPage
      hasPreviousPage
      startCursor
      endCursor
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    page_info = result["data"]["branches"]["pageInfo"]
    print("\n📄 Pagination Info:")
    print(f"  - Has Next Page: {page_info['hasNextPage']}")
    print(f"  - Has Previous Page: {page_info['hasPreviousPage']}")
    print(f"  - End Cursor: {page_info['endCursor']}\n")
    
    df = display_branches(result)
    display(df)

## Test 5: Filter by City

In [ ]:
# Query branches in Mumbai
query = """
query {
  branches(first: 10, filter: {city: "MUMBAI"}) {
    edges {
      node {
        ifsc
        branch
        address
        city
        bank {
          name
        }
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    total = result["data"]["branches"]["totalCount"]
    print(f"\n✅ Found {total:,} branches in MUMBAI\n")
    
    df = display_branches(result)
    display(df)

## Test 6: Filter by State

In [ ]:
# Query branches in Maharashtra
query = """
query {
  branches(first: 10, filter: {state: "MAHARASHTRA"}) {
    edges {
      node {
        ifsc
        branch
        city
        district
        bank {
          name
        }
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    total = result["data"]["branches"]["totalCount"]
    print(f"\n✅ Found {total:,} branches in MAHARASHTRA\n")
    
    df = display_branches(result)
    display(df)

## Test 7: Get Single Branch by IFSC

In [ ]:
# Get specific branch by IFSC code
query = """
query {
  branch(ifsc: "ABHY0065001") {
    ifsc
    branch
    address
    city
    district
    state
    bank {
      id
      name
    }
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result and result["data"]["branch"]:
    branch = result["data"]["branch"]
    print("\n✅ Branch Details:\n")
    print(f"  IFSC Code: {branch['ifsc']}")
    print(f"  Branch: {branch['branch']}")
    print(f"  Bank: {branch['bank']['name']}")
    print(f"  Address: {branch['address']}")
    print(f"  City: {branch['city']}")
    print(f"  District: {branch['district']}")
    print(f"  State: {branch['state']}")

## Test 8: Query All Banks

In [ ]:
# Get list of banks
query = """
query {
  banks(first: 20) {
    edges {
      node {
        id
        name
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    banks = result["data"]["banks"]["edges"]
    total = result["data"]["banks"]["totalCount"]
    
    print(f"\n✅ Total Banks: {total:,}\n")
    
    # Create DataFrame
    bank_list = [{"ID": edge["node"]["id"], "Bank Name": edge["node"]["name"]} for edge in banks]
    df = pd.DataFrame(bank_list)
    display(df)

## Test 9: Filter by Bank Name

In [ ]:
# Get branches of a specific bank
query = """
query {
  branches(first: 10, filter: {bankName: "STATE BANK"}) {
    edges {
      node {
        ifsc
        branch
        city
        bank {
          name
        }
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    total = result["data"]["branches"]["totalCount"]
    print(f"\n✅ Found {total:,} branches matching 'STATE BANK'\n")
    
    df = display_branches(result)
    display(df)

## Test 10: Complex Query with Multiple Filters

In [ ]:
# Combine multiple filters
query = """
query {
  branches(first: 10, filter: {city: "MUMBAI", state: "MAHARASHTRA"}) {
    edges {
      node {
        ifsc
        branch
        address
        city
        district
        bank {
          name
        }
      }
    }
    totalCount
  }
}
"""

result = run_graphql_query(query)

if result and "data" in result:
    total = result["data"]["branches"]["totalCount"]
    print(f"\n✅ Found {total:,} branches in MUMBAI, MAHARASHTRA\n")
    
    df = display_branches(result)
    display(df)

## Test 11: Data Analysis - Branches by State

In [ ]:
# Collect branch counts by state (sample)
states = ["MAHARASHTRA", "KARNATAKA", "TAMIL NADU", "DELHI", "WEST BENGAL"]
state_counts = []

for state in states:
    query = f"""
    query {{
      branches(first: 1, filter: {{state: "{state}"}}) {{
        totalCount
      }}
    }}
    """
    result = run_graphql_query(query)
    if result and "data" in result:
        count = result["data"]["branches"]["totalCount"]
        state_counts.append({"State": state, "Branch Count": count})

# Display results
df_states = pd.DataFrame(state_counts)
print("\n📊 Branch Distribution by State:\n")
display(df_states)

# Simple bar chart
df_states.plot(x='State', y='Branch Count', kind='bar', figsize=(10, 5))

## Summary

This notebook demonstrated:
1. ✅ Health checks and API status
2. ✅ Basic GraphQL queries
3. ✅ Pagination with cursors
4. ✅ Filtering by city, state, bank name
5. ✅ Single record queries by IFSC
6. ✅ Complex multi-filter queries
7. ✅ Data analysis and visualization

## Next Steps

- Try your own custom queries
- Explore the GraphiQL interface at `http://localhost:8000/graphql`
- Check API documentation at `http://localhost:8000/docs`

Happy querying! 🚀